<a href="https://colab.research.google.com/github/Rafia-Shaikh-eng/A-Filter-based-Feature-Selection-Approach-in-Multilabel-Classification/blob/main/Fisher_Score_Feature_selection_50_MLC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

importing required libraries

In [ ]:
# if the scikit-multilearn not installed run this cell
!pip install scikit-multilearn
# most of the datasets in Weka Arff format for this installed this library
!pip install arff

Created 03 arrays of each measuring metrics for 03 classifier

In [ ]:
#03 classifiers hamming loss list
hamming_loss_MLTSVM=[]
hamming_loss_MLKNN=[]
hamming_loss_BRKNNa=[]
#03 classifier accuracy list
accuracy_MLTSVM=[]
accuracy_MLKNN=[]
accuracy_BRKNNa=[]
#03 classifier precision list
precision_MLTSVM=[]
precision_MLKNN=[]
precision_BRKNNa=[]
#03 classifier recall list
recall_MLTSVM=[]
recall_MLKNN=[]
recall_BRKNNa=[]
#03 classifier f1 measure list
f1_measure_MLTSVM=[]
f1_measure_MLKNN=[]
f1_measure_BRKNNa=[]

# 01 Medical Dataset

In [ ]:
#Load Medical Dataset
from skmultilearn.dataset import load_dataset
x,y, _, _ = load_dataset('medical', 'undivided')
print("Actual Dataset")
print(x.shape,y.shape)
#change to matrix
x1=x.todense()
y1=y.todense()
import pandas as pd
#change to dataFrame
x1=pd.DataFrame(x1)
y1=pd.DataFrame(y1)
print("Coverted into Dataframe")
print(x1.shape,y1.shape)

medical:undivided - exists, not redownloading
Actual Dataset
(978, 1449) (978, 45)
Coverted into Dataframe
(978, 1449) (978, 45)


In [ ]:
int(x1.shape[1]*0.2)

289

## Feature Selection - Fisher Score

In [ ]:
import numpy as np
from sklearn.feature_selection import f_classif

def calculate_fisher_score(X, Y):
    fisher_scores, _ = f_classif(X, Y)
    return fisher_scores

def select_top_K_features(scores, K):
    sorted_indices = np.argsort(scores)[::-1]
    return sorted_indices[:K]

def select_features(X, Y, K,p):
    selected_features = set()
    MI = {}  # Create an empty dictionary

    # Calculate Fisher's scores for each feature associated with each label
    for label in Y.columns:
        MI[label] = calculate_fisher_score(X, Y[label])
    # Select top K features for each label
    for label in MI.keys():
        topK = select_top_K_features(MI[label], K)
        MI[label] = topK
    # Count occurrences of selected features
    feature_counts = {}
    for label in MI.keys():
        for feature in MI[label]:
            if feature in feature_counts:
                feature_counts[feature] += 1
            else:
                feature_counts[feature] = 1

    # Select features based on the aggregated counts
    for feature, count in feature_counts.items():
        if count >= int(len((Y.columns))*p):  # Adjust this condition as needed
            selected_features.add(feature)

    return selected_features


In [ ]:
#call funtion
# Usage example
selected_features = select_features(x1, y1, K=int((x1.shape[1])*0.5),p=0.5)
# print("Selected features based on Fisher's score:", selected_features)
#updated the original x
for i in list(x1.columns):
    if i not in selected_features:
        x1 = x1.drop(columns=[i])

C:\Users\Aashir\anaconda3\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:116: RuntimeWarning: divide by zero encountered in divide
  f = msb / msw
C:\Users\Aashir\anaconda3\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:116: RuntimeWarning: divide by zero encountered in divide
  f = msb / msw
C:\Users\Aashir\anaconda3\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:116: RuntimeWarning: divide by zero encountered in divide
  f = msb / msw
C:\Users\Aashir\anaconda3\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:116: RuntimeWarning: divide by zero encountered in divide
  f = msb / msw
C:\Users\Aashir\anaconda3\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:116: RuntimeWarning: divide by zero encountered in divide
  f = msb / msw
C:\Users\Aashir\anaconda3\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:116: RuntimeWarning: divide by zero encountered in divide
  

In [ ]:
x1.shape

(978, 726)

## MLC

In [ ]:
from scipy import sparse as s
x1 = s.csr_matrix(x1)
y1 = s.csr_matrix(y1)
from skmultilearn.model_selection import iterative_train_test_split
x1_train, y1_train, x1_test, y1_test = iterative_train_test_split(x1, y1, test_size = 0.3)
print(x1_train.shape, x1_test.shape, y1_train.shape, y1_test.shape)

(685, 726) (293, 726) (685, 45) (293, 45)


### MLTSVM

In [ ]:
#MLTSVM
from skmultilearn.adapt import MLTSVM
classifier1 = MLTSVM(max_iteration=1000,c_k=2**-6)
# train
classifier1.fit(x1_train, y1_train)
# predict
MLTSVM1_predictions = classifier1.predict(x1_test)

### MLKNN

In [ ]:
#MLKNN
from skmultilearn.adapt import MLkNN
#MLKNN is an adaptation of the kNN lazy learning algorithm for multi-label data.
# apply MLkNN for medical dataset it has 45 labels.
classifier2 = MLkNN(k=45)
# train
classifier2.fit(x1_train, y1_train)
# predict
MLKNN1_predictions = classifier2.predict(x1_test)

C:\Users\Aashir\anaconda3\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass n_neighbors=45 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


### BRKNNa

In [ ]:
from skmultilearn.adapt import BRkNNaClassifier
classifier3 = BRkNNaClassifier(k=45)
# train
classifier3.fit(x1_train, y1_train)
# predict
BR1_predictions = classifier3.predict(x1_test)

## Measuring Metrics

In [ ]:
import sklearn.metrics as m
# #Hamming Loss
# hamming_loss_MLTSVM.append(round(m.hamming_loss(y1_test, MLTSVM1_predictions),2))
# hamming_loss_MLKNN.append(round(m.hamming_loss(y1_test, MLKNN1_predictions),2))
# hamming_loss_BRKNNa.append(round(m.hamming_loss(y1_test, BR1_predictions),2))

# #Accuracy
# accuracy_MLTSVM.append(round(m.accuracy_score(y1_test, MLTSVM1_predictions),2))
# accuracy_MLKNN.append(round(m.accuracy_score(y1_test, MLKNN1_predictions),2))
# accuracy_BRKNNa.append(round(m.accuracy_score(y1_test, BR1_predictions),2))

#Micro Precision
precision_MLTSVM.append(round(m.precision_score(y1_test, MLTSVM1_predictions, average='micro', zero_division='warn'),2))
precision_MLKNN.append(round(m.precision_score(y1_test, MLKNN1_predictions, average='micro', zero_division='warn'),2))
precision_BRKNNa.append(round(m.precision_score(y1_test, BR1_predictions, average='micro', zero_division='warn'),2))

#Micro Recall
recall_MLTSVM.append(round(m.recall_score(y1_test, MLTSVM1_predictions, average='micro',zero_division='warn'),2))
recall_MLKNN.append(round(m.recall_score(y1_test, MLKNN1_predictions, average='micro',zero_division='warn'),2))
recall_BRKNNa.append(round(m.recall_score(y1_test, BR1_predictions, average='micro',zero_division='warn'),2))
# #Micro F1 measure
# f1_measure_MLTSVM.append(round(m.f1_score(y1_test, MLTSVM1_predictions, average='micro',zero_division='warn'),2))
# f1_measure_MLKNN.append(round(m.f1_score(y1_test, MLKNN1_predictions, average='micro',zero_division='warn'),2))
# f1_measure_BRKNNa.append(round(m.f1_score(y1_test, BR1_predictions, average='micro',zero_division='warn'),2))

In [ ]:
# print("------------------------------")
# print("\t(Examples,Labels)")
# print("------------------------------")
# print("MLTSVM -> ",MLTSVM1_predictions.shape)
# print("MLKNN  -> ",MLKNN1_predictions.shape)
# print("BRKNNa -> ",BR1_predictions.shape)
# import sklearn.metrics as m
# print("------------------------------")
# print("\tHamming Loss")
# print("------------------------------")
# print("MLTSVM -> ",round(m.hamming_loss(y1_test, MLTSVM1_predictions),2))
# print("MLKNN  -> ",round(m.hamming_loss(y1_test, MLKNN1_predictions),2))
# print("BRKNNa -> ",round(m.hamming_loss(y1_test, BR1_predictions),2))
# print("------------------------------")
# print("\tAccuracy")
# print("------------------------------")
# print("MLTSVM -> ",round(m.accuracy_score(y1_test, MLTSVM1_predictions),2))
# print("MLKNN  -> ",round(m.accuracy_score(y1_test, MLKNN1_predictions),2))
# print("BRKNNa -> ",round(m.accuracy_score(y1_test, BR1_predictions),2))
# print("------------------------------")
print("\tPrecision")
print("------------------------------")
print("MLTSVM -> ",round(m.precision_score(y1_test, MLTSVM1_predictions, average='micro', zero_division='warn'),2))
print("MLKNN -> ",round(m.precision_score(y1_test, MLKNN1_predictions, average='micro', zero_division='warn'),2))
print("BRKNNa -> ",round(m.precision_score(y1_test, BR1_predictions, average='micro', zero_division='warn'),2))
print("------------------------------")
print("\tRecall")
print("------------------------------")
print("MLTSVM -> ",round(m.recall_score(y1_test, MLTSVM1_predictions, average='micro',zero_division='warn'),2))
print("MLTSVM -> ",round(m.recall_score(y1_test, MLKNN1_predictions, average='micro',zero_division='warn'),2))
print("MLTSVM -> ",round(m.recall_score(y1_test, BR1_predictions, average='micro',zero_division='warn'),2))
print("------------------------------")
# print("\tF1-measure")
# print("------------------------------")
# #f1=2*(p*r)/(p+r)
# print("MLTSVM -> ",round(m.f1_score(y1_test, MLTSVM1_predictions, average='micro',zero_division='warn'),2))
# print("MLTSVM -> ",round(m.f1_score(y1_test, MLKNN1_predictions, average='micro',zero_division='warn'),2))
# print("MLTSVM -> ",round(m.f1_score(y1_test, BR1_predictions, average='micro',zero_division='warn'),2))
# print()

	Precision
------------------------------
MLTSVM ->  0.76
MLKNN ->  0.84
BRKNNa ->  0.81
------------------------------
	Recall
------------------------------
MLTSVM ->  0.68
MLTSVM ->  0.49
MLTSVM ->  0.3
------------------------------


# 02 Enron Dataset

In [ ]:
from skmultilearn.dataset import load_dataset
x,y, _, _ = load_dataset('enron', 'undivided')
print("Actual Dataset")
print(x.shape,y.shape)
#change to matrix
x2=x.todense()
y2=y.todense()
import pandas as pd
#change to dataFrame
x2=pd.DataFrame(x2)
y2=pd.DataFrame(y2)
print("Coverted into Dataframe")
print(x2.shape,y2.shape)

enron:undivided - exists, not redownloading
Actual Dataset
(1702, 1001) (1702, 53)
Coverted into Dataframe
(1702, 1001) (1702, 53)


## Feature Selection - Fisher Score

In [ ]:
#call funtion
# Usage example
selected_features = select_features(x2, y2, K=int((x2.shape[1])*0.5),p=0.5)
# print("Selected features based on Fisher's score:", selected_features)
#updated the original x
for i in list(x2.columns):
    if i not in selected_features:
        x2 = x2.drop(columns=[i])

In [ ]:
x2.shape

(1702, 568)

## MLC

In [ ]:
from scipy import sparse as s
x2 = s.csr_matrix(x2)
y2 = s.csr_matrix(y2)
from skmultilearn.model_selection import iterative_train_test_split
x2_train, y2_train, x2_test, y2_test = iterative_train_test_split(x2, y2, test_size = 0.3)
print(x2_train.shape, x2_test.shape, y2_train.shape, y2_test.shape)

(1182, 568) (520, 568) (1182, 53) (520, 53)


### MLTSVM

In [ ]:
#from skmultilearn.adapt import MLTSVM
classifier1 = MLTSVM(c_k =2**-6)
# train
classifier1.fit(x2_train, y2_train)
# predict
MLTSVM2_predictions = classifier1.predict(x2_test)

### MLKNN

In [ ]:
from skmultilearn.adapt import MLkNN
#MLKNN is an adaptation of the kNN lazy learning algorithm for multi-label data.
# apply MLkNN for enron dataset it has 53 labels.
classifier2 = MLkNN(k=53)

# train
classifier2.fit(x2_train, y2_train)

# predict
MLKNN2_predictions = classifier2.predict(x2_test)

C:\Users\Aashir\anaconda3\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass n_neighbors=53 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


### BRKNNa

In [ ]:
from skmultilearn.adapt import BRkNNaClassifier
classifier3 = BRkNNaClassifier(k=53)
# train
classifier3.fit(x2_train, y2_train)
# predict
BR2_predictions = classifier3.predict(x2_test)

## Measuring Metrics

In [ ]:
import sklearn.metrics as m
# #Hamming Loss
# hamming_loss_MLTSVM.append(round(m.hamming_loss(y2_test, MLTSVM2_predictions),2))
# hamming_loss_MLKNN.append(round(m.hamming_loss(y2_test, MLKNN2_predictions),2))
# hamming_loss_BRKNNa.append(round(m.hamming_loss(y2_test, BR2_predictions),2))

# #Accuracy
# accuracy_MLTSVM.append(round(m.accuracy_score(y2_test, MLTSVM2_predictions),2))
# accuracy_MLKNN.append(round(m.accuracy_score(y2_test, MLKNN2_predictions),2))
# accuracy_BRKNNa.append(round(m.accuracy_score(y2_test, BR2_predictions),2))

#Micro Precision
precision_MLTSVM.append(round(m.precision_score(y2_test, MLTSVM2_predictions, average='micro', zero_division='warn'),2))
precision_MLKNN.append(round(m.precision_score(y2_test, MLKNN2_predictions, average='micro', zero_division='warn'),2))
precision_BRKNNa.append(round(m.precision_score(y2_test, BR2_predictions, average='micro', zero_division='warn'),2))

#Micro Recall
recall_MLTSVM.append(round(m.recall_score(y2_test, MLTSVM2_predictions, average='micro',zero_division='warn'),2))
recall_MLKNN.append(round(m.recall_score(y2_test, MLKNN2_predictions, average='micro',zero_division='warn'),2))
recall_BRKNNa.append(round(m.recall_score(y2_test, BR2_predictions, average='micro',zero_division='warn'),2))
# #Micro F1 measure
# f1_measure_MLTSVM.append(round(m.f1_score(y2_test, MLTSVM2_predictions, average='micro',zero_division='warn'),2))
# f1_measure_MLKNN.append(round(m.f1_score(y2_test, MLKNN2_predictions, average='micro',zero_division='warn'),2))
# f1_measure_BRKNNa.append(round(m.f1_score(y2_test, BR2_predictions, average='micro',zero_division='warn'),2))

In [ ]:
# print("------------------------------")
# print("\t(Examples,Labels)")
# print("------------------------------")
# print("MLTSVM -> ",MLTSVM2_predictions.shape)
# print("MLKNN  -> ",MLKNN2_predictions.shape)
# print("BRKNNa -> ",BR2_predictions.shape)
# import sklearn.metrics as m
# print("------------------------------")
# print("\tHamming Loss")
# print("------------------------------")
# print("MLTSVM -> ",round(m.hamming_loss(y2_test, MLTSVM2_predictions),2))
# print("MLKNN  -> ",round(m.hamming_loss(y2_test, MLKNN2_predictions),2))
# print("BRKNNa -> ",round(m.hamming_loss(y2_test, BR2_predictions),2))
# print("------------------------------")
# print("\tAccuracy")
# print("------------------------------")
# print("MLTSVM -> ",round(m.accuracy_score(y2_test, MLTSVM2_predictions),2))
# print("MLKNN  -> ",round(m.accuracy_score(y2_test, MLKNN2_predictions),2))
# print("BRKNNa -> ",round(m.accuracy_score(y2_test, BR2_predictions),2))
# print("------------------------------")
print("\tPrecision")
print("------------------------------")
print("MLTSVM -> ",round(m.precision_score(y2_test, MLTSVM2_predictions, average='micro', zero_division='warn'),2))
print("MLKNN -> ",round(m.precision_score(y2_test, MLKNN2_predictions, average='micro', zero_division='warn'),2))
print("BRKNNa -> ",round(m.precision_score(y2_test, BR2_predictions, average='micro', zero_division='warn'),2))
print("------------------------------")
print("\tRecall")
print("------------------------------")
print("MLTSVM -> ",round(m.recall_score(y2_test, MLTSVM2_predictions, average='micro',zero_division='warn'),2))
print("MLTSVM -> ",round(m.recall_score(y2_test, MLKNN2_predictions, average='micro',zero_division='warn'),2))
print("MLTSVM -> ",round(m.recall_score(y2_test, BR2_predictions, average='micro',zero_division='warn'),2))
# print("------------------------------")
# print("\tF1-measure")
# print("------------------------------")
# #f1=2*(p*r)/(p+r)
# print("MLTSVM -> ",round(m.f1_score(y2_test, MLTSVM2_predictions, average='micro',zero_division='warn'),2))
# print("MLTSVM -> ",round(m.f1_score(y2_test, MLKNN2_predictions, average='micro',zero_division='warn'),2))
# print("MLTSVM -> ",round(m.f1_score(y2_test, BR2_predictions, average='micro',zero_division='warn'),2))
# print()

	Precision
------------------------------
MLTSVM ->  0.43
MLKNN ->  0.62
BRKNNa ->  0.58
------------------------------
	Recall
------------------------------
MLTSVM ->  0.41
MLTSVM ->  0.29
MLTSVM ->  0.11


# 03 Scene Dataset

In [ ]:
#from skmultilearn.dataset import load_dataset
x3,y3, _, _ = load_dataset('scene', 'undivided')
print("Actual Dataset")
print(x3.shape,y3.shape)
#change to matrix
x3=x3.todense()
y3=y3.todense()
import pandas as pd
#change to dataFrame
x3=pd.DataFrame(x3)
y3=pd.DataFrame(y3)
print("Coverted into Dataframe")
print(x3.shape,y3.shape)

scene:undivided - exists, not redownloading
Actual Dataset
(2407, 294) (2407, 6)
Coverted into Dataframe
(2407, 294) (2407, 6)


## Feature Selection - Fisher Score

In [ ]:
#call funtion
# Usage example
selected_features = select_features(x3, y3, K=int((x3.shape[1])*0.5),p=0.5)
# print("Selected features based on Fisher's score:", selected_features)
#updated the original x
for i in list(x3.columns):
    if i not in selected_features:
        x3 = x3.drop(columns=[i])

In [ ]:
x3.shape

(2407, 164)

## MLC

In [ ]:
from scipy import sparse as s
x3 = s.csr_matrix(x3)
y3 = s.csr_matrix(y3)
from skmultilearn.model_selection import iterative_train_test_split
x3_train, y3_train, x3_test, y3_test = iterative_train_test_split(x3, y3, test_size = 0.3)
print(x3_train.shape, x3_test.shape, y3_train.shape, y3_test.shape)

(1685, 164) (722, 164) (1685, 6) (722, 6)


### MLTSVM

In [ ]:
#from skmultilearn.adapt import MLTSVM
classifier1 = MLTSVM(c_k =2**-6)
# train
classifier1.fit(x3_train, y3_train)
# predict
MLTSVM3_predictions = classifier1.predict(x3_test)

### MLKNN

In [ ]:
#from skmultilearn.adapt import MLkNN
#MLKNN is an adaptation of the kNN lazy learning algorithm for multi-label data.
# apply MLkNN for Scene dataset it has 6 labels.
classifier2 = MLkNN(k=6)

# train
classifier2.fit(x3_train, y3_train)

# predict
MLKNN3_predictions = classifier2.predict(x3_test)

C:\Users\Aashir\anaconda3\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass n_neighbors=6 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


### BRKNNa

In [ ]:
from skmultilearn.adapt import BRkNNaClassifier
classifier3 = BRkNNaClassifier(k=6)
# train
classifier3.fit(x3_train, y3_train)
# predict
BR3_predictions = classifier3.predict(x3_test)

## Measuring Metrics

In [ ]:
import sklearn.metrics as m
# #Hamming Loss
# hamming_loss_MLTSVM.append(round(m.hamming_loss(y3_test, MLTSVM3_predictions),2))
# hamming_loss_MLKNN.append(round(m.hamming_loss(y3_test, MLKNN3_predictions),2))
# hamming_loss_BRKNNa.append(round(m.hamming_loss(y3_test, BR3_predictions),2))

# #Accuracy
# accuracy_MLTSVM.append(round(m.accuracy_score(y3_test, MLTSVM3_predictions),2))
# accuracy_MLKNN.append(round(m.accuracy_score(y3_test, MLKNN3_predictions),2))
# accuracy_BRKNNa.append(round(m.accuracy_score(y3_test, BR3_predictions),2))

#Micro Precision
precision_MLTSVM.append(round(m.precision_score(y3_test, MLTSVM3_predictions, average='micro', zero_division='warn'),2))
precision_MLKNN.append(round(m.precision_score(y3_test, MLKNN3_predictions, average='micro', zero_division='warn'),2))
precision_BRKNNa.append(round(m.precision_score(y3_test, BR3_predictions, average='micro', zero_division='warn'),2))

#Micro Recall
recall_MLTSVM.append(round(m.recall_score(y3_test, MLTSVM3_predictions, average='micro',zero_division='warn'),2))
recall_MLKNN.append(round(m.recall_score(y3_test, MLKNN3_predictions, average='micro',zero_division='warn'),2))
recall_BRKNNa.append(round(m.recall_score(y3_test, BR3_predictions, average='micro',zero_division='warn'),2))
# #Micro F1 measure
# f1_measure_MLTSVM.append(round(m.f1_score(y3_test, MLTSVM3_predictions, average='micro',zero_division='warn'),2))
# f1_measure_MLKNN.append(round(m.f1_score(y3_test, MLKNN3_predictions, average='micro',zero_division='warn'),2))
# f1_measure_BRKNNa.append(round(m.f1_score(y3_test, BR3_predictions, average='micro',zero_division='warn'),2))

In [ ]:
# print("------------------------------")
# print("\t(Examples,Labels")
# print("------------------------------")
# print("MLTSVM -> ",MLTSVM3_predictions.shape)
# print("MLKNN  -> ",MLKNN3_predictions.shape)
# print("BRKNNa -> ",BR3_predictions.shape)
# import sklearn.metrics as m
# print("------------------------------")
# print("\tHamming Loss")
# print("------------------------------")
# print("MLTSVM -> ",round(m.hamming_loss(y3_test, MLTSVM3_predictions),2))
# print("MLKNN  -> ",round(m.hamming_loss(y3_test, MLKNN3_predictions),2))
# print("BRKNNa -> ",round(m.hamming_loss(y3_test, BR3_predictions),2))
# print("------------------------------")
# print("\tAccuracy")
# print("------------------------------")
# print("MLTSVM -> ",round(m.accuracy_score(y3_test, MLTSVM3_predictions),2))
# print("MLKNN  -> ",round(m.accuracy_score(y3_test, MLKNN3_predictions),2))
# print("BRKNNa -> ",round(m.accuracy_score(y3_test, BR3_predictions),2))
# print("------------------------------")
print("\tPrecision")
print("------------------------------")
print("MLTSVM -> ",round(m.precision_score(y3_test, MLTSVM3_predictions, average='micro',zero_division='warn'),2))
print("MLKNN -> ",round(m.precision_score(y3_test, MLKNN3_predictions, average='micro',zero_division='warn'),2))
print("BRKNNa -> ",round(m.precision_score(y3_test, BR3_predictions, average='micro',zero_division='warn'),2))
print("------------------------------")
print("\tRecall")
print("------------------------------")
print("MLTSVM -> ",round(m.recall_score(y3_test, MLTSVM3_predictions, average='micro',zero_division='warn'),2))
print("MLKNN -> ",round(m.recall_score(y3_test, MLKNN3_predictions, average='micro',zero_division='warn'),2))
print("BRKNNa -> ",round(m.recall_score(y3_test, BR3_predictions, average='micro',zero_division='warn'),2))
# print("------------------------------")
# print("\tF1-measure")
# print("------------------------------")
# #f1=2*(p*r)/(p+r)
# print("MLTSVM -> ",round(m.f1_score(y3_test, MLTSVM3_predictions, average='micro',zero_division='warn'),2))
# print("MLKNN -> ",round(m.f1_score(y3_test, MLKNN3_predictions, average='micro',zero_division='warn'),2))
# print("BRKNNa -> ",round(m.f1_score(y3_test, BR3_predictions, average='micro',zero_division='warn'),2))
# print()

	Precision
------------------------------
MLTSVM ->  0.18
MLKNN ->  0.8
BRKNNa ->  0.83
------------------------------
	Recall
------------------------------
MLTSVM ->  1.0
MLKNN ->  0.64
BRKNNa ->  0.58


# 04 Emotions Dataset

In [ ]:
from skmultilearn.dataset import load_dataset
x4,y4, _, _ = load_dataset('emotions', 'undivided')
print("Actual Dataset")
print(x4.shape,y4.shape)
#change to matrix
x4=x4.todense()
y4=y4.todense()
import pandas as pd
#change to dataFrame
x4=pd.DataFrame(x4)
y4=pd.DataFrame(y4)
print("Coverted into Dataframe")
print(x4.shape,y4.shape)

emotions:undivided - exists, not redownloading
Actual Dataset
(593, 72) (593, 6)
Coverted into Dataframe
(593, 72) (593, 6)


## Feature Selection - Fisher Score

In [ ]:
#call funtion
# Usage example
selected_features = select_features(x4, y4, K=int((x4.shape[1])*0.5),p=0.5)
# print("Selected features based on Fisher's score:", selected_features)
#updated the original x
for i in list(x4.columns):
    if i not in selected_features:
        x4 = x4.drop(columns=[i])

In [ ]:
x4.shape

(593, 40)

## MLC

In [ ]:
from scipy import sparse as s
x4 = s.csr_matrix(x4)
y4 = s.csr_matrix(y4)
from skmultilearn.model_selection import iterative_train_test_split
x4_train, y4_train, x4_test, y4_test = iterative_train_test_split(x4, y4, test_size = 0.3)
print(x4_train.shape, x4_test.shape, y4_train.shape, y4_test.shape)

(416, 40) (177, 40) (416, 6) (177, 6)


### MLTSVM

In [ ]:
from skmultilearn.adapt import MLTSVM
classifier1 = MLTSVM(c_k =2**-6)
# train
classifier1.fit(x4_train, y4_train)
# predict
MLTSVM5_predictions = classifier1.predict(x4_test)

### MLKNN

In [ ]:
from skmultilearn.adapt import MLkNN
#MLKNN is an adaptation of the kNN lazy learning algorithm for multi-label data.

# apply MLkNN for delicious dataset it has 983 labels.
classifier2 = MLkNN(k=27)

# train
classifier2.fit(x4_train, y4_train)

# predict
MLKNN5_predictions = classifier2.predict(x4_test)

C:\Users\Aashir\anaconda3\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass n_neighbors=27 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


### BRKNNa

In [ ]:
from skmultilearn.adapt import BRkNNaClassifier
classifier3 = BRkNNaClassifier(k=27)
# train
classifier3.fit(x4_train, y4_train)
# predict
BR5_predictions = classifier3.predict(x4_test)

## Measuring Metrics

In [ ]:
import sklearn.metrics as m
# #Hamming Loss
# hamming_loss_MLTSVM.append(round(m.hamming_loss(y4_test, MLTSVM5_predictions),2))
# hamming_loss_MLKNN.append(round(m.hamming_loss(y4_test, MLKNN5_predictions),2))
# hamming_loss_BRKNNa.append(round(m.hamming_loss(y4_test, BR5_predictions),2))

# #Accuracy
# accuracy_MLTSVM.append(round(m.accuracy_score(y4_test, MLTSVM5_predictions),2))
# accuracy_MLKNN.append(round(m.accuracy_score(y4_test, MLKNN5_predictions),2))
# accuracy_BRKNNa.append(round(m.accuracy_score(y4_test, BR5_predictions),2))

#Micro Precision
precision_MLTSVM.append(round(m.precision_score(y4_test, MLTSVM5_predictions, average='micro', zero_division='warn'),2))
precision_MLKNN.append(round(m.precision_score(y4_test, MLKNN5_predictions, average='micro', zero_division='warn'),2))
precision_BRKNNa.append(round(m.precision_score(y4_test, BR5_predictions, average='micro', zero_division='warn'),2))

#Micro Recall
recall_MLTSVM.append(round(m.recall_score(y4_test, MLTSVM5_predictions, average='micro',zero_division='warn'),2))
recall_MLKNN.append(round(m.recall_score(y4_test, MLKNN5_predictions, average='micro',zero_division='warn'),2))
recall_BRKNNa.append(round(m.recall_score(y4_test, BR5_predictions, average='micro',zero_division='warn'),2))
# #Micro F1 measure
# f1_measure_MLTSVM.append(round(m.f1_score(y4_test, MLTSVM5_predictions, average='micro',zero_division='warn'),2))
# f1_measure_MLKNN.append(round(m.f1_score(y4_test, MLKNN5_predictions, average='micro',zero_division='warn'),2))
# f1_measure_BRKNNa.append(round(m.f1_score(y4_test, BR5_predictions, average='micro',zero_division='warn'),2))

In [ ]:
# print("------------------------------")
# print("\t(Examples,Labels")
# print("------------------------------")
# print("MLTSVM -> ",MLTSVM5_predictions.shape)
# print("MLKNN  -> ",MLKNN5_predictions.shape)
# print("BRKNNa -> ",BR5_predictions.shape)
# import sklearn.metrics as m
# print("------------------------------")
# print("\tHamming Loss")
# print("------------------------------")
# print("MLTSVM -> ",round(m.hamming_loss(y4_test, MLTSVM5_predictions),2))
# print("MLKNN  -> ",round(m.hamming_loss(y4_test, MLKNN5_predictions),2))
# print("BRKNNa -> ",round(m.hamming_loss(y4_test, BR5_predictions),2))
# print("------------------------------")
# print("\tAccuracy")
# print("------------------------------")
# print("MLTSVM -> ",round(m.accuracy_score(y4_test, MLTSVM5_predictions),2))
# print("MLKNN  -> ",round(m.accuracy_score(y4_test, MLKNN5_predictions),2))
# print("BRKNNa -> ",round(m.accuracy_score(y4_test, BR5_predictions),2))
# print("------------------------------")
print("\tPrecision")
print("------------------------------")
print("MLTSVM -> ",round(m.precision_score(y4_test, MLTSVM5_predictions, average='micro',zero_division='warn'),2))
print("MLKNN -> ",round(m.precision_score(y4_test, MLKNN5_predictions, average='micro',zero_division='warn'),2))
print("MLKNNa -> ",round(m.precision_score(y4_test, BR5_predictions, average='micro',zero_division='warn'),2))
print("------------------------------")
print("\tRecall")
print("------------------------------")
print("MLTSVM -> ",round(m.recall_score(y4_test, MLTSVM5_predictions, average='micro',zero_division='warn'),2))
print("MLKNN -> ",round(m.recall_score(y4_test, MLKNN5_predictions, average='micro',zero_division='warn'),2))
print("BRKNNa -> ",round(m.recall_score(y4_test, BR5_predictions, average='micro',zero_division='warn'),2))
print("------------------------------")
# print("\tF1-measure")
# print("------------------------------")
# #f1=2*(p*r)/(p+r)
# print("MLTSVM -> ",round(m.f1_score(y4_test, MLTSVM5_predictions, average='micro',zero_division='warn'),2))
# print("MLKNN -> ",round(m.f1_score(y4_test, MLKNN5_predictions, average='micro',zero_division='warn'),2))
# print("BRKNNa -> ",round(m.f1_score(y4_test, BR5_predictions, average='micro',zero_division='warn'),2))
# print()

	Precision
------------------------------
MLTSVM ->  0.32
MLKNN ->  0.62
MLKNNa ->  0.67
------------------------------
	Recall
------------------------------
MLTSVM ->  1.0
MLKNN ->  0.49
BRKNNa ->  0.5
------------------------------


# 05 Genbase Dataset

In [ ]:
from skmultilearn.dataset import load_dataset
x5,y5, _, _ = load_dataset('genbase', 'undivided')
print("Actual Dataset")
print(x5.shape,y5.shape)
#change to matrix
x5=x5.todense()
y5=y5.todense()
import pandas as pd
#change to dataFrame
x5=pd.DataFrame(x5)
y5=pd.DataFrame(y5)
print("Coverted into Dataframe")
print(x5.shape,y5.shape)

genbase:undivided - exists, not redownloading
Actual Dataset
(662, 1186) (662, 27)
Coverted into Dataframe
(662, 1186) (662, 27)


## Feature Selection - Fisher Score

In [ ]:
#call funtion
# Usage example
selected_features = select_features(x5, y5, K=int((x5.shape[1])*0.5),p=0.5)
# print("Selected features based on Fisher's score:", selected_features)
#updated the original x
for i in list(x5.columns):
    if i not in selected_features:
        x5 = x5.drop(columns=[i])

C:\Users\Aashir\anaconda3\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:114: UserWarning: Features [   1    2    3 ... 1183 1184 1185] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx,
C:\Users\Aashir\anaconda3\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:116: RuntimeWarning: divide by zero encountered in divide
  f = msb / msw
C:\Users\Aashir\anaconda3\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:116: RuntimeWarning: invalid value encountered in divide
  f = msb / msw
C:\Users\Aashir\anaconda3\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:114: UserWarning: Features [   1    2    3 ... 1183 1184 1185] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx,
C:\Users\Aashir\anaconda3\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:116: RuntimeWarning: divide by zero encountered in divide
  f = msb / msw
C:\Users\Aas

In [ ]:
x5.shape

(662, 593)

## MLC

In [ ]:
from scipy import sparse as s
x5 = s.csr_matrix(x5)
y5 = s.csr_matrix(y5)
from skmultilearn.model_selection import iterative_train_test_split
x5_train, y5_train, x5_test, y5_test = iterative_train_test_split(x5, y5, test_size = 0.3)
print(x5_train.shape, x5_test.shape, y5_train.shape, y5_test.shape)

(463, 593) (199, 593) (463, 27) (199, 27)


### MLTSVM

In [ ]:
from skmultilearn.adapt import MLTSVM
classifier1 = MLTSVM(c_k =2**-6)
# train
classifier1.fit(x5_train, y5_train)
# predict
MLTSVM5_predictions = classifier1.predict(x5_test)

### MLKNN

In [ ]:
from skmultilearn.adapt import MLkNN
#MLKNN is an adaptation of the kNN lazy learning algorithm for multi-label data.

# apply MLkNN for delicious dataset it has 983 labels.
classifier2 = MLkNN(k=27)

# train
classifier2.fit(x5_train, y5_train)

# predict
MLKNN5_predictions = classifier2.predict(x5_test)

C:\Users\Aashir\anaconda3\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass n_neighbors=27 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


### BRKNNa

In [ ]:
from skmultilearn.adapt import BRkNNaClassifier
classifier3 = BRkNNaClassifier(k=27)
# train
classifier3.fit(x5_train, y5_train)
# predict
BR5_predictions = classifier3.predict(x5_test)

## Measuring Metrics

In [ ]:
import sklearn.metrics as m
# #Hamming Loss
# hamming_loss_MLTSVM.append(round(m.hamming_loss(y5_test, MLTSVM5_predictions),2))
# hamming_loss_MLKNN.append(round(m.hamming_loss(y5_test, MLKNN5_predictions),2))
# hamming_loss_BRKNNa.append(round(m.hamming_loss(y5_test, BR5_predictions),2))

# #Accuracy
# accuracy_MLTSVM.append(round(m.accuracy_score(y5_test, MLTSVM5_predictions),2))
# accuracy_MLKNN.append(round(m.accuracy_score(y5_test, MLKNN5_predictions),2))
# accuracy_BRKNNa.append(round(m.accuracy_score(y5_test, BR5_predictions),2))

#Micro Precision
precision_MLTSVM.append(round(m.precision_score(y5_test, MLTSVM5_predictions, average='micro', zero_division='warn'),2))
precision_MLKNN.append(round(m.precision_score(y5_test, MLKNN5_predictions, average='micro', zero_division='warn'),2))
precision_BRKNNa.append(round(m.precision_score(y5_test, BR5_predictions, average='micro', zero_division='warn'),2))

#Micro Recall
recall_MLTSVM.append(round(m.recall_score(y5_test, MLTSVM5_predictions, average='micro',zero_division='warn'),2))
recall_MLKNN.append(round(m.recall_score(y5_test, MLKNN5_predictions, average='micro',zero_division='warn'),2))
recall_BRKNNa.append(round(m.recall_score(y5_test, BR5_predictions, average='micro',zero_division='warn'),2))
# #Micro F1 measure
# f1_measure_MLTSVM.append(round(m.f1_score(y5_test, MLTSVM5_predictions, average='micro',zero_division='warn'),2))
# f1_measure_MLKNN.append(round(m.f1_score(y5_test, MLKNN5_predictions, average='micro',zero_division='warn'),2))
# f1_measure_BRKNNa.append(round(m.f1_score(y5_test, BR5_predictions, average='micro',zero_division='warn'),2))

C:\Users\Aashir\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# print("------------------------------")
# print("\t(Examples,Labels")
# print("------------------------------")
# print("MLTSVM -> ",MLTSVM5_predictions.shape)
# print("MLKNN  -> ",MLKNN5_predictions.shape)
# print("BRKNNa -> ",BR5_predictions.shape)
# import sklearn.metrics as m
# print("------------------------------")
# print("\tHamming Loss")
# print("------------------------------")
# print("MLTSVM -> ",round(m.hamming_loss(y5_test, MLTSVM5_predictions),2))
# print("MLKNN  -> ",round(m.hamming_loss(y5_test, MLKNN5_predictions),2))
# print("BRKNNa -> ",round(m.hamming_loss(y5_test, BR5_predictions),2))
# print("------------------------------")
# print("\tAccuracy")
# print("------------------------------")
# print("MLTSVM -> ",round(m.accuracy_score(y5_test, MLTSVM5_predictions),2))
# print("MLKNN  -> ",round(m.accuracy_score(y5_test, MLKNN5_predictions),2))
# print("BRKNNa -> ",round(m.accuracy_score(y5_test, BR5_predictions),2))
# print("------------------------------")
print("\tPrecision")
print("------------------------------")
print("MLTSVM -> ",round(m.precision_score(y5_test, MLTSVM5_predictions, average='micro',zero_division='warn'),2))
print("MLKNN -> ",round(m.precision_score(y5_test, MLKNN5_predictions, average='micro',zero_division='warn'),2))
print("MLKNNa -> ",round(m.precision_score(y5_test, BR5_predictions, average='micro',zero_division='warn'),2))
print("------------------------------")
print("\tRecall")
print("------------------------------")
print("MLTSVM -> ",round(m.recall_score(y5_test, MLTSVM5_predictions, average='micro',zero_division='warn'),2))
print("MLKNN -> ",round(m.recall_score(y5_test, MLKNN5_predictions, average='micro',zero_division='warn'),2))
print("BRKNNa -> ",round(m.recall_score(y5_test, BR5_predictions, average='micro',zero_division='warn'),2))
# print("------------------------------")
# print("\tF1-measure")
# print("------------------------------")
# #f1=2*(p*r)/(p+r)
# print("MLTSVM -> ",round(m.f1_score(y5_test, MLTSVM5_predictions, average='micro',zero_division='warn'),2))
# print("MLKNN -> ",round(m.f1_score(y5_test, MLKNN5_predictions, average='micro',zero_division='warn'),2))
# print("BRKNNa -> ",round(m.f1_score(y5_test, BR5_predictions, average='micro',zero_division='warn'),2))
# print()

	Precision
------------------------------
MLTSVM ->  0.05
MLKNN ->  0.0
MLKNNa ->  0.0
------------------------------
	Recall
------------------------------
MLTSVM ->  1.0
MLKNN ->  0.0
BRKNNa ->  0.0


C:\Users\Aashir\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Results based on  EVALUATION MEASURES

##Hamming Loss

In [ ]:
import matplotlib.pyplot as plt
dataset=['medical','enron','scene','gebase']
hl=[]
print(hamming_loss_MLKNN)
fig, ax=plt.subplots()
ax.plot(dataset,hamming_loss_MLTSVM, label='MLTSVM',marker="H")
ax.plot(dataset,hamming_loss_MLKNN, label='MLKNN',marker=">")
ax.plot(dataset,hamming_loss_BRKNNa, label='BRKNNa',marker="*")
ax.legend(loc='upper left')
ax.set_title('Chi2 for 20% FS')
ax.set_xlabel('Datasets')
ax.set_ylabel('Hamming Loss')
plt.tight_layout()
plt.savefig('chi-20-hamming loss')

##Accuracy

In [ ]:
#import matplotlib.pyplot as plt
fig, ax=plt.subplots()
ax.plot(dataset,accuracy_MLTSVM, label='MLTSVM',marker="H")
ax.plot(dataset,accuracy_MLKNN, label='MLKNN',marker='>')
ax.plot(dataset,accuracy_BRKNNa, label='BRKNNa',marker='*')
ax.legend(loc='upper left')
ax.set_title('Chi2 for 20% FS')
ax.set_xlabel('Datasets')
ax.set_ylabel('Accuracy')
plt.savefig('chi-20-Accuracy')

##Precision

In [ ]:
fig, ax=plt.subplots()
ax.plot(dataset,precision_MLTSVM, label='MLTSVM',marker="H")
ax.plot(dataset,precision_MLKNN, label='MLKNN',marker='>')
ax.plot(dataset,precision_BRKNNa, label='BRKNNa',marker='*')
ax.legend(loc='lower left')
ax.set_title('Chi2 for 20% FS')
ax.set_xlabel('Datasets')
ax.set_ylabel('Micro Precision')
plt.tight_layout()
plt.savefig('chi-20-Precision')

##Recall

In [ ]:
fig, ax=plt.subplots()
ax.plot(dataset,recall_MLTSVM, label='MLTSVM',marker="H")
ax.plot(dataset,recall_MLKNN, label='MLKNN',marker='>')
ax.plot(dataset,recall_BRKNNa, label='BRKNNa',marker='*')
ax.legend(loc='upper left')
ax.set_title('Chi2 for 20% FS')
ax.set_xlabel('Datasets')
ax.set_ylabel('Micro Recall')
plt.tight_layout()
plt.savefig('chi-20-Recall')

##F1 Measure

In [ ]:
#F1 measure
fig, ax=plt.subplots()
ax.plot(dataset,f1_measure_MLTSVM, label='MLTSVM',marker="H")
ax.plot(dataset,f1_measure_MLKNN, label='MLKNN',marker='>')
ax.plot(dataset,f1_measure_BRKNNa, label='BRKNNa',marker='*')
ax.legend(loc='lower left')
ax.set_title('Chi2 for 20% FS')
ax.set_xlabel('Datasets')
ax.set_ylabel('Micro F1 measure')
plt.tight_layout()
plt.savefig('chi-20-f1 measure')

In [ ]:
#F1 measure
fig, ax=plt.subplots()
HL=[0.02, ]
ax.plot(dataset,f1_measure_MLTSVM, label='MLTSVM',marker="H")
ax.plot(dataset,f1_measure_MLKNN, label='MLKNN',marker='>')
ax.plot(dataset,f1_measure_BRKNNa, label='BRKNNa',marker='*')
ax.legend(loc='lower left')
ax.set_title('Chi2 for 20% FS')
ax.set_xlabel('Datasets')
ax.set_ylabel('Micro F1 measure')
plt.tight_layout()
plt.savefig('chi-20-f1 measure')